# Reporte de promesas acumuladas en el mes

In [1]:
import pandas as pd
import numpy as np
import mysql.connector
from mysql.connector import Error
from datetime import date, datetime
import ConeccionesSucursales as cs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
#Definimos la ruta en donde queremos guardar los archivos
mes2 = 'Enero2021'
ruta = '/home/estadistico/Documents/Erick/'+mes2+'/Reportes diarios/Promesas mes'
ruta_pagos = '/home/estadistico/Documents/Erick'
#Definimos la ruta donde estan las asignaciones
ruta_asig_baz = '/home/estadistico/Documents/Erick/Banco Azteca/Asignacion csv'
mes = 20210101

In [3]:
#Agregamos las variables a ocupar
servidor = '192.168.15.12'
puerto = int('3306')
usuario = 'estadisticas'
contraseña = 'estadisticas8474'
base = 'procesos_externos'

In [4]:
#Asignamos valores a los parametros \n",
today = date.today().strftime('%Y%m%d')
now = datetime.now().strftime('%d-%m-%Y-%H:%M')

In [5]:
#Hacemos la conexion con el servidor\n",
try:
    conn = mysql.connector.connect(user=usuario,
                               password=contraseña,
                               host=servidor,
                               port=3306,
                               database=base)
    conn.set_charset_collation('latin1')
except mysql.Error as e:
    print("Failed to execute stored procedure: {}".format(error))

cursor = conn.cursor()

# Liverpool

In [6]:
#Hacemos la consulta referente a las gestiones de Liverpool
gestion_liv = cs.query_naucalpan('liverpool_rpt_gestiones_detallado',mes,today)
gestion_liv.columns = ['folio_gestion','firma_id','unegocio_id','credito','nombre_credito','telefono','tipo_telefono','fecha_gestion','usuario','nombre_usuario','dictamen','accion','resultado','fecha_promesa','monto_promesa','comentarios']
gestion_liv = gestion_liv.loc[gestion_liv['dictamen']=='PROMESA']

In [7]:
#Asignacion Liverpool
asignacion_liv = pd.read_csv(''+ruta_pagos+'/Liverpool/Asignacion/Asignaciones csv/Asignacion '+mes2+'.csv',converters={'# CUENTA':str})
asignacion_liv = asignacion_liv[['ID AGENCIA', '# CUENTA']]
asignacion_liv.columns = ['division','credito']

In [8]:
#Hacemos el consolidado 
consolidado_liv = pd.merge(gestion_liv,asignacion_liv,how='left',on=['credito'])
cols = ['monto_promesa','credito']
consolidado_liv[cols] = consolidado_liv[cols].apply(pd.to_numeric, errors='coerce')
consolidado_liv = consolidado_liv[(consolidado_liv['monto_promesa']>50)]
consolidado_liv = consolidado_liv[(consolidado_liv['monto_promesa']<500000)]
#consolidado_liv.head(3)

In [9]:
#Obtenemos datos de las promesas de Liverpool
consolidado_liv2 = consolidado_liv.drop_duplicates(subset=['credito','fecha_promesa','monto_promesa'])
pivot_liv = pd.pivot_table(consolidado_liv2,index=['division'],values=['credito','monto_promesa'],aggfunc=['count',np.sum],margins=True)
pivot_liv = pd.DataFrame(pivot_liv.to_records())
pivot_liv = pivot_liv.iloc[:,[0,1,4]]
pivot_liv.columns = ['Division','NumeroPromesas','SumaPromesas']
pivot_liv.fillna(0,inplace=True)
#pivot_liv[['NumeroPromesas','PromedioPromesa','SumaPromesas']] = pivot_liv[['NumeroPromesas','PromedioPromesa','SumaPromesas']].applymap("{0:.2f}".format)
pivot_liv

,Division,NumeroPromesas,SumaPromesas
0,DKN00REM,240,1216869.74
1,DKN02REM,239,880990.72
2,DKN04REM,131,583782.44
3,SA001REM,225,346023.32
4,SA002REM,11,22817.00
5,All,846,3050483.22


# Bradesco 

In [10]:
#Hacemos la consulta referente a las gestiones de Bradesco
gestion_brad = cs.query_naucalpan('bradescard_rpt_gestiones_detallado',mes,today)
gestion_brad.columns = ['folio_gestion','firma_id','unegocio_id','NUMERO_TARJETA','nombre_credito','telefono','tipo_telefono','fecha_gestion','usuario','nombre_usuario','dictamen','accion','resultado','accion_resultado','fecha_promesa','monto_promesa','comentarios']
gestion_brad = gestion_brad.loc[gestion_brad['dictamen']=='PROMESA']
gestion_brad['NUMERO_TARJETA'] = pd.to_numeric(gestion_brad['NUMERO_TARJETA']) 

In [11]:
asignacion_brad_c0 = pd.read_csv(''+ruta_pagos+'/Bradesco/Asignacion_c0/CSV/ASIGNACION C0 '+mes2+'.csv')
asignacion_brad_c0 = asignacion_brad_c0.iloc[:,[0,2,3,8,10,11,14,25]]
asignacion_brad_extra = pd.read_csv(''+ruta_pagos+'/Bradesco/Asignacion_extra/CSV/Asignacion Extra '+mes2+'.csv',encoding= 'unicode_escape')
asignacion_brad_extra = asignacion_brad_extra.iloc[:,[0,2,3,5,9,25,26,28]]
asignacion_brad_extra.columns = ['CODIGO_ORG','NUMERO_CUENTA','NUMERO_TARJETA','RFC','ESTADO','SALDO_TOTAL','PAGO_DESCUENTO','PAGOSVENCIDOS']

asignacion_brad = pd.concat([asignacion_brad_c0,asignacion_brad_extra],ignore_index=True)

In [12]:
#Hacemos la union de asignacion y promesas
consolidado_brad = pd.merge(gestion_brad,asignacion_brad,how='left',on=['NUMERO_TARJETA'])
cols = ['monto_promesa']
consolidado_brad[cols] = consolidado_brad[cols].apply(pd.to_numeric, errors='coerce')
consolidado_brad = consolidado_brad[(consolidado_brad['monto_promesa']>50)]
consolidado_brad = consolidado_brad[(consolidado_brad['monto_promesa']<500000)]

In [13]:
#Obtenemos datos de las promesas de Bradesco
pivot_brad = pd.pivot_table(consolidado_brad,index=['PAGOSVENCIDOS'],values=['NUMERO_TARJETA','monto_promesa'],aggfunc=['count',np.sum],margins=True)
pivot_brad = pd.DataFrame(pivot_brad.to_records())
pivot_brad = pivot_brad.iloc[:,[0,1,4]]
pivot_brad.columns = ['Division','NumeroPromesas','SumaPromesas']
pivot_brad.fillna(0,inplace=True)
#pivot_brad[['NumeroPromesas','PromedioPromesa','SumaPromesas']] = pivot_gestion_brad[['NumeroPromesas','PromedioPromesa','SumaPromesas']].applymap("{0:.2f}".format)
pivot_brad

,Division,NumeroPromesas,SumaPromesas
0,A,44,78995.00
1,B,9,18565.00
2,C0 4PV,461,707382.59
3,C0 5PV,254,419754.00
4,C0 6PV,170,287608.00
5,C1,253,464956.00
6,C2,223,352595.00
7,C3,143,179705.00
8,C4,68,85425.00
9,All,1625,2594985.59


# Credifiel

In [14]:
#Hacemos la consulta referente a las gestiones de Credifiel
gestion_cred = cs.query_naucalpan('credifiel_rpt_gestiones_detallado',mes,today)
gestion_cred.columns = ['folio_gestion','unegocio_id','fecha_gestion','hora_gestion','credito','nombre_credito','telefono','usuario','nombre_usuario','accion','resultado','fecha_promesa','monto_promesa','comentarios','accion_credifiel','resultado_credifiel','dictamen']
gestion_cred = gestion_cred.loc[gestion_cred['accion_credifiel']=='PDP']
cols = ['monto_promesa','unegocio_id']
gestion_cred[cols] = gestion_cred[cols].apply(pd.to_numeric, errors='coerce')
gestion_cred = gestion_cred.loc[gestion_cred['monto_promesa']>0]

In [15]:
#Hacemos la union de asignacion y promesas
consolidado_cred = gestion_cred
cols = ['monto_promesa','unegocio_id']
consolidado_cred[cols] = consolidado_cred[cols].apply(pd.to_numeric, errors='coerce')
consolidado_cred = consolidado_cred[(consolidado_cred['monto_promesa']>50)]
consolidado_cred = consolidado_cred[(consolidado_cred['monto_promesa']<500000)]
consolidado_cred['division'] = 'EXTRAJUDICIAL'

In [16]:
#Obtenemos datos de las promesas de Credifiel
pivot_cred = pd.pivot_table(consolidado_cred,index=['division'],values=['unegocio_id','monto_promesa'],aggfunc=('count',np.sum))
pivot_cred = pd.DataFrame(pivot_cred.to_records())
pivot_cred = pivot_cred.iloc[:,[0,3,2]]
pivot_cred.columns = ['Division','NumeroPromesas','SumaPromesas']
pivot_cred.fillna(0,inplace=True)
pivot_cred = pivot_cred.loc[pivot_cred['SumaPromesas']>0]
#pivot_gestion_cred[['NumeroPromesas','PromedioPromesa','SumaPromesas']] = pivot_gestion_brad[['NumeroPromesas','PromedioPromesa','SumaPromesas']].applymap("{0:.2f}".format)
pivot_cred

,Division,NumeroPromesas,SumaPromesas
0,EXTRAJUDICIAL,111,309724.74


# Banco Azteca

In [17]:
#Hacemos la consulta referente a las gestiones de Banco Azteca
gestion_baz = cs.query_naucalpan('baz_rpt_gestiones_detallado',mes,today)
gestion_baz.columns = ['folio_gestion','firma_id','unegocio_id','credito','nombre_credito','telefono','tipo_telefono','fecha_gestion','usuario','nombre_usuario','dictamen','accion_resultado','fecha_promesa','monto_promesa','comentarios']
gestion_baz = gestion_baz.loc[gestion_baz['dictamen']=='PROMESA']

In [18]:
consolidado_baz = gestion_baz
cols = ['monto_promesa']
consolidado_baz[cols] = consolidado_baz[cols].apply(pd.to_numeric, errors='coerce')
consolidado_baz = consolidado_baz[(consolidado_baz['monto_promesa']>50)]
consolidado_baz = consolidado_baz[(consolidado_baz['monto_promesa']<500000)]

In [19]:
#Obtenemos datos de las promesas de Banco Azteca
consolidado_baz2 = consolidado_baz.drop_duplicates(subset=['credito','fecha_promesa','monto_promesa'])
pivot_baz = pd.pivot_table(consolidado_baz,index=['dictamen'],values=['unegocio_id','monto_promesa'],aggfunc=('count',np.sum))
pivot_baz = pd.DataFrame(pivot_baz.to_records())
pivot_baz = pivot_baz.iloc[:,[0,3,2]]
pivot_baz.columns = ['Dictamen','NumeroPromesas','SumaPromesas']
pivot_baz.fillna(0,inplace=True)
pivot_baz

,Dictamen,NumeroPromesas,SumaPromesas
0,PROMESA,1976,1777571.0


In [20]:
writer = pd.ExcelWriter(''+ruta+'/Reporte Promesas Acumulado Mes '+now+'.xlsx', engine='xlsxwriter')
pivot_liv.to_excel(writer,'PromMesLiverpool',index=False,header=True)
pivot_brad.to_excel(writer,'PromMesBradesco',index=False,header=True)
pivot_cred.to_excel(writer,'PromMesCredifiel',index=False,header=True)
pivot_baz.to_excel(writer,'PromBancoAzteca',index=False,header=True)

writer.save()
writer.close()